In [4]:
import warnings

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV

from sklearn.ensemble import VotingClassifier

from datetime import datetime
from datetime import timedelta


pd.set_option('display.max_columns', None)  # display all columns
# data = pd.read_csv("data.csv")
data = pd.read_csv("C:\\Users\\Rom\\Desktop\\data.csv")

data.sort_values('game_date', inplace=True)

################added###########
data["shot_id_sub"] = range(1, len(data) + 1)
################################


data.set_index('shot_id_sub', inplace=True)

data["action_type"] = data["action_type"].astype('object')
data["combined_shot_type"] = data["combined_shot_type"].astype('category')
data["game_event_id"] = data["game_event_id"].astype('category')
data["game_id"] = data["game_id"].astype('category')
data["period"] = data["period"].astype('object')
#data["playoffs"] = data["playoffs"].astype('category')
data["season"] = data["season"].astype('category')
data["shot_made_flag"] = data["shot_made_flag"].astype('category')
data["shot_type"] = data["shot_type"].astype('category')
data["team_id"] = data["team_id"].astype('category')

data.describe(include=['object', 'category'])

##############################dealing with the data###########################################
unknown_mask = data['shot_made_flag'].isnull()
data_cl = data.copy()  # create a copy of data frame
target = data_cl['shot_made_flag'].copy()

data_cl.drop('team_id', axis=1, inplace=True)  # Always one number
data_cl.drop('lat', axis=1, inplace=True)  # Correlated with loc_x
data_cl.drop('lon', axis=1, inplace=True)  # Correlated with loc_y
data_cl.drop('game_id', axis=1, inplace=True)  # Independent
data_cl.drop('game_event_id', axis=1, inplace=True)  # Independent
data_cl.drop('team_name', axis=1, inplace=True)  # Always LA Lakers
data_cl.drop('shot_made_flag', axis=1, inplace=True)
data_cl.drop('playoffs', axis=1, inplace=True)

############
## Remaining time
data_cl['seconds_from_period_end'] = 60 * data_cl['minutes_remaining'] + data_cl['seconds_remaining']
data_cl['last_5_sec_in_period'] = data_cl['seconds_from_period_end'] < 5
data_cl['last_5_sec_in_period']=data_cl['last_5_sec_in_period'].astype('int')

data_cl['secondsFromPeriodStart'] = 60*(11-data_cl['minutes_remaining'])+(60-data_cl['seconds_remaining'])
data_cl['secondsFromGameStart']   = (data['period'] <= 4).astype(int)*(data_cl['period']-1)*12*60 + (data_cl['period'] > 4).astype(int)*((data_cl['period']-4)*5*60 + 3*12*60) + data_cl['secondsFromPeriodStart']

data_cl.drop('minutes_remaining', axis=1, inplace=True)
data_cl.drop('seconds_remaining', axis=1, inplace=True)
data_cl.drop('seconds_from_period_end', axis=1, inplace=True)

## Matchup - (away/home)
data_cl['home_play'] = data_cl['matchup'].str.contains('vs').astype('int')
data_cl.drop('matchup', axis=1, inplace=True)
# Game date
x =  data_cl.game_date.str.replace('-','')
data_cl['yesterday'] = (pd.to_datetime(x, format='%Y%m%d')- timedelta(days=1)) 
data_cl['back2back'] = data_cl.yesterday.isin(data_cl.game_date).astype('int')
data_cl.drop('yesterday', axis=1, inplace=True) 

data_cl['game_date'] = pd.to_datetime(data_cl['game_date'])
data_cl['game_year'] = data_cl['game_date'].dt.year
data_cl['game_month'] = data_cl['game_date'].dt.month
#data_cl['dayOfWeek']=data_cl['game_date'].dt.dayofweek

data_cl.drop('game_date', axis=1, inplace=True)

data_cl['3pt'] = data_cl['shot_type'].str.contains('3PT').astype('int')
data_cl.drop('shot_type', axis=1, inplace=True)

# Replace 20 least common action types with value 'Other'
rare_action_types = data_cl['action_type'].value_counts().sort_values().index.values[:20]
data_cl.loc[data_cl['action_type'].isin(rare_action_types), 'action_type'] = 'Other'

data_cl.drop('season', axis=1, inplace=True)
data_cl.drop('shot_distance', axis=1, inplace=True)
#data_cl.drop('loc_y', axis=1, inplace=True)
data_cl.drop('opponent', axis=1, inplace=True)

# ENCODE CATEGORIAL VARIABLES
categorial_cols = [
    'action_type', 'combined_shot_type', 'period',
    'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
    'game_month','game_year']
# 'loc_x', 'loc_y' ,'opponent' 

for cc in categorial_cols:
    dummies = pd.get_dummies(data_cl[cc])
    dummies = dummies.add_prefix("{}#".format(cc))
    data_cl.drop(cc, axis=1, inplace=True)
    data_cl = data_cl.join(dummies)

# taking down features
data_submit = data_cl[unknown_mask]
##################creating the model##################
X = data_cl[~unknown_mask]
Y = target[~unknown_mask]
seed = 7
processors=1
num_folds=3
num_instances=len(X)
scoring='log_loss'

kfold = KFold(n=num_instances, n_folds=num_folds, random_state=seed)

             
lr_grid = GridSearchCV(estimator = LogisticRegression(random_state=seed),param_grid = {'penalty': [ 'l2'],'solver':['newton-cg', 'lbfgs', 'sag', 'saga']
    }, cv = kfold,scoring = scoring, n_jobs = processors)
lr_grid.fit(X, Y)
#print(lr_grid.best_score_)
#print(lr_grid.best_params_)
 #Create sub models
estimators = []

estimators.append(('lr', LogisticRegression(solver='newton-cg',penalty='l2', C=1)))
estimators.append(('dtc', DecisionTreeClassifier( max_depth=4, random_state=seed)))
estimators.append(('lda', LinearDiscriminantAnalysis( solver='svd', n_components=None)))

# create the ensemble model
ensemble = VotingClassifier(estimators, voting='soft', weights=[6,3,1])

results = cross_val_score(ensemble, X, Y, cv=kfold, scoring=scoring,n_jobs=processors)
#print("({0:.3f}) +/- ({1:.3f})".format(results.mean(), results.std()))

############for submissiom##############
submissionFinal = pd.DataFrame()

# the for should run from here and on
counter = 0;
for index, row in data_submit.iterrows():
    counter += 1
    if (len(range(index)) > 100):
      #  if (len(range(index)) > 5000):
       #     X = data_cl[(index-2000):index]
        #    Y = target[(index-2000):index]
        #else:
         #   X = data_cl[:index]
          #  Y = target[:index]
        X = data_cl[:index]
        Y = target[:index]
        # take the known data
        X = X[~unknown_mask]
        Y = Y[~unknown_mask]
        
        ensemble.fit(X, Y)
        preds =  ensemble.predict_proba(data_cl[(index-1):index])
        ##tunning
        #lr=LogisticRegression()
        #lr.fit(X, Y)
        #preds =  lr.predict_proba(data_cl[(index-1):index])
        
        #lda=LinearDiscriminantAnalysis()
        #lda.fit(X, Y)
        #preds1 =  lda.predict_proba(data_cl[(index-1):index])
        
        #dt=DecisionTreeClassifier(max_depth=4,random_state=0)
        #dt.fit(X, Y)
        #preds2 =dt.predict_proba(data_cl[(index-1):index])
        
        #tmp=0.7*preds+0.2*preds1+0.1*preds;
        
        submission = pd.DataFrame()
        submission["shot_id"] = data_submit[(counter - 1):counter]["shot_id"]

        submission["shot_made_flag"] = preds[:, 1]
        frames = [submissionFinal, submission]
        result = pd.concat(frames)
        submissionFinal = result
    else:
        submission = pd.DataFrame()
        submission["shot_id"] = data_submit[(counter - 1):counter]["shot_id"]

        submission["shot_made_flag"] = 0.446161#kobe pg%
        frames = [submissionFinal, submission]
        result = pd.concat(frames)
        submissionFinal = result

    print("the counter is : {} ".format(counter))
#  print("the shot id is {}".format( data_submit[(counter-1):counter]["shot_id"]))

result.to_csv("res_21.csv", index=False)

the counter is : 1 
the counter is : 2 
the counter is : 3 
the counter is : 4 
the counter is : 5 
the counter is : 6 
the counter is : 7 
the counter is : 8 
the counter is : 9 
the counter is : 10 
the counter is : 11 
the counter is : 12 
the counter is : 13 
the counter is : 14 
the counter is : 15 
the counter is : 16 
the counter is : 17 
the counter is : 18 
the counter is : 19 
the counter is : 20 
the counter is : 21 
the counter is : 22 
the counter is : 23 
the counter is : 24 
the counter is : 25 
the counter is : 26 
the counter is : 27 
the counter is : 28 
the counter is : 29 
the counter is : 30 
the counter is : 31 
the counter is : 32 
the counter is : 33 
the counter is : 34 
the counter is : 35 
the counter is : 36 
the counter is : 37 
the counter is : 38 
the counter is : 39 
the counter is : 40 
the counter is : 41 
the counter is : 42 
the counter is : 43 
the counter is : 44 
the counter is : 45 
the counter is : 46 
the counter is : 47 
the counter is : 48 
t

the counter is : 379 
the counter is : 380 
the counter is : 381 
the counter is : 382 
the counter is : 383 
the counter is : 384 
the counter is : 385 
the counter is : 386 
the counter is : 387 
the counter is : 388 
the counter is : 389 
the counter is : 390 
the counter is : 391 
the counter is : 392 
the counter is : 393 
the counter is : 394 
the counter is : 395 
the counter is : 396 
the counter is : 397 
the counter is : 398 
the counter is : 399 
the counter is : 400 
the counter is : 401 
the counter is : 402 
the counter is : 403 
the counter is : 404 
the counter is : 405 
the counter is : 406 
the counter is : 407 
the counter is : 408 
the counter is : 409 
the counter is : 410 
the counter is : 411 
the counter is : 412 
the counter is : 413 
the counter is : 414 
the counter is : 415 
the counter is : 416 
the counter is : 417 
the counter is : 418 
the counter is : 419 
the counter is : 420 
the counter is : 421 
the counter is : 422 
the counter is : 423 
the counte

the counter is : 752 
the counter is : 753 
the counter is : 754 
the counter is : 755 
the counter is : 756 
the counter is : 757 
the counter is : 758 
the counter is : 759 
the counter is : 760 
the counter is : 761 
the counter is : 762 
the counter is : 763 
the counter is : 764 
the counter is : 765 
the counter is : 766 
the counter is : 767 
the counter is : 768 
the counter is : 769 
the counter is : 770 
the counter is : 771 
the counter is : 772 
the counter is : 773 
the counter is : 774 
the counter is : 775 
the counter is : 776 
the counter is : 777 
the counter is : 778 
the counter is : 779 
the counter is : 780 
the counter is : 781 
the counter is : 782 
the counter is : 783 
the counter is : 784 
the counter is : 785 
the counter is : 786 
the counter is : 787 
the counter is : 788 
the counter is : 789 
the counter is : 790 
the counter is : 791 
the counter is : 792 
the counter is : 793 
the counter is : 794 
the counter is : 795 
the counter is : 796 
the counte

the counter is : 1119 
the counter is : 1120 
the counter is : 1121 
the counter is : 1122 
the counter is : 1123 
the counter is : 1124 
the counter is : 1125 
the counter is : 1126 
the counter is : 1127 
the counter is : 1128 
the counter is : 1129 
the counter is : 1130 
the counter is : 1131 
the counter is : 1132 
the counter is : 1133 
the counter is : 1134 
the counter is : 1135 
the counter is : 1136 
the counter is : 1137 
the counter is : 1138 
the counter is : 1139 
the counter is : 1140 
the counter is : 1141 
the counter is : 1142 
the counter is : 1143 
the counter is : 1144 
the counter is : 1145 
the counter is : 1146 
the counter is : 1147 
the counter is : 1148 
the counter is : 1149 
the counter is : 1150 
the counter is : 1151 
the counter is : 1152 
the counter is : 1153 
the counter is : 1154 
the counter is : 1155 
the counter is : 1156 
the counter is : 1157 
the counter is : 1158 
the counter is : 1159 
the counter is : 1160 
the counter is : 1161 
the counter

the counter is : 1476 
the counter is : 1477 
the counter is : 1478 
the counter is : 1479 
the counter is : 1480 
the counter is : 1481 
the counter is : 1482 
the counter is : 1483 
the counter is : 1484 
the counter is : 1485 
the counter is : 1486 
the counter is : 1487 
the counter is : 1488 
the counter is : 1489 
the counter is : 1490 
the counter is : 1491 
the counter is : 1492 
the counter is : 1493 
the counter is : 1494 
the counter is : 1495 
the counter is : 1496 
the counter is : 1497 
the counter is : 1498 
the counter is : 1499 
the counter is : 1500 
the counter is : 1501 
the counter is : 1502 
the counter is : 1503 
the counter is : 1504 
the counter is : 1505 
the counter is : 1506 
the counter is : 1507 
the counter is : 1508 
the counter is : 1509 
the counter is : 1510 
the counter is : 1511 
the counter is : 1512 
the counter is : 1513 
the counter is : 1514 
the counter is : 1515 
the counter is : 1516 
the counter is : 1517 
the counter is : 1518 
the counter

...

the counter is : 3261 
the counter is : 3262 
the counter is : 3263 
the counter is : 3264 
the counter is : 3265 
the counter is : 3266 
the counter is : 3267 
the counter is : 3268 
the counter is : 3269 
the counter is : 3270 
the counter is : 3271 
the counter is : 3272 
the counter is : 3273 
the counter is : 3274 
the counter is : 3275 
the counter is : 3276 
the counter is : 3277 
the counter is : 3278 
the counter is : 3279 
the counter is : 3280 
the counter is : 3281 
the counter is : 3282 
the counter is : 3283 
the counter is : 3284 
the counter is : 3285 
the counter is : 3286 
the counter is : 3287 
the counter is : 3288 
the counter is : 3289 
the counter is : 3290 
the counter is : 3291 
the counter is : 3292 
the counter is : 3293 
the counter is : 3294 
the counter is : 3295 
the counter is : 3296 
the counter is : 3297 
the counter is : 3298 
the counter is : 3299 
the counter is : 3300 
the counter is : 3301 
the counter is : 3302 
the counter is : 3303 
the counter

the counter is : 3618 
the counter is : 3619 
the counter is : 3620 
the counter is : 3621 
the counter is : 3622 
the counter is : 3623 
the counter is : 3624 
the counter is : 3625 
the counter is : 3626 
the counter is : 3627 
the counter is : 3628 
the counter is : 3629 
the counter is : 3630 
the counter is : 3631 
the counter is : 3632 
the counter is : 3633 
the counter is : 3634 
the counter is : 3635 
the counter is : 3636 
the counter is : 3637 
the counter is : 3638 
the counter is : 3639 
the counter is : 3640 
the counter is : 3641 
the counter is : 3642 
the counter is : 3643 
the counter is : 3644 
the counter is : 3645 
the counter is : 3646 
the counter is : 3647 
the counter is : 3648 
the counter is : 3649 
the counter is : 3650 
the counter is : 3651 
the counter is : 3652 
the counter is : 3653 
the counter is : 3654 
the counter is : 3655 
the counter is : 3656 
the counter is : 3657 
the counter is : 3658 
the counter is : 3659 
the counter is : 3660 
the counter

the counter is : 3975 
the counter is : 3976 
the counter is : 3977 
the counter is : 3978 
the counter is : 3979 
the counter is : 3980 
the counter is : 3981 
the counter is : 3982 
the counter is : 3983 
the counter is : 3984 
the counter is : 3985 
the counter is : 3986 
the counter is : 3987 
the counter is : 3988 
the counter is : 3989 
the counter is : 3990 
the counter is : 3991 
the counter is : 3992 
the counter is : 3993 
the counter is : 3994 
the counter is : 3995 
the counter is : 3996 
the counter is : 3997 
the counter is : 3998 
the counter is : 3999 
the counter is : 4000 
the counter is : 4001 
the counter is : 4002 
the counter is : 4003 
the counter is : 4004 
the counter is : 4005 
the counter is : 4006 
the counter is : 4007 
the counter is : 4008 
the counter is : 4009 
the counter is : 4010 
the counter is : 4011 
the counter is : 4012 
the counter is : 4013 
the counter is : 4014 
the counter is : 4015 
the counter is : 4016 
the counter is : 4017 
the counter

the counter is : 4332 
the counter is : 4333 
the counter is : 4334 
the counter is : 4335 
the counter is : 4336 
the counter is : 4337 
the counter is : 4338 
the counter is : 4339 
the counter is : 4340 
the counter is : 4341 
the counter is : 4342 
the counter is : 4343 
the counter is : 4344 
the counter is : 4345 
the counter is : 4346 
the counter is : 4347 
the counter is : 4348 
the counter is : 4349 
the counter is : 4350 
the counter is : 4351 
the counter is : 4352 
the counter is : 4353 
the counter is : 4354 
the counter is : 4355 
the counter is : 4356 
the counter is : 4357 
the counter is : 4358 
the counter is : 4359 
the counter is : 4360 
the counter is : 4361 
the counter is : 4362 
the counter is : 4363 
the counter is : 4364 
the counter is : 4365 
the counter is : 4366 
the counter is : 4367 
the counter is : 4368 
the counter is : 4369 
the counter is : 4370 
the counter is : 4371 
the counter is : 4372 
the counter is : 4373 
the counter is : 4374 
the counter

the counter is : 4689 
the counter is : 4690 
the counter is : 4691 
the counter is : 4692 
the counter is : 4693 
the counter is : 4694 
the counter is : 4695 
the counter is : 4696 
the counter is : 4697 
the counter is : 4698 
the counter is : 4699 
the counter is : 4700 
the counter is : 4701 
the counter is : 4702 
the counter is : 4703 
the counter is : 4704 
the counter is : 4705 
the counter is : 4706 
the counter is : 4707 
the counter is : 4708 
the counter is : 4709 
the counter is : 4710 
the counter is : 4711 
the counter is : 4712 
the counter is : 4713 
the counter is : 4714 
the counter is : 4715 
the counter is : 4716 
the counter is : 4717 
the counter is : 4718 
the counter is : 4719 
the counter is : 4720 
the counter is : 4721 
the counter is : 4722 
the counter is : 4723 
the counter is : 4724 
the counter is : 4725 
the counter is : 4726 
the counter is : 4727 
the counter is : 4728 
the counter is : 4729 
the counter is : 4730 
the counter is : 4731 
the counter

In [5]:
result.to_csv("last_run.csv", index=False)

# code with the methodes

In [ ]:
print ("lila tov")

# decision tree